In [ ]:
import torch
from transformers import ViTModel, ViTFeatureExtractor

device = 'cuda' if torch.cuda.is_available() else 'cpu'

feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k").to(device)
model.eval()


: 

In [ ]:
outputs = model(pixel_values=inputs, output_attentions=True)
attentions = outputs.attentions  # A tuple/list of attention layers

In [ ]:
from PIL import Image
import requests

url = "https://example.com/your_image.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
inputs = feature_extractor(images=image, return_tensors="pt").to(device)


In [ ]:
cls_attention = attentions[l][0, h, 0, 1:]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

def visualize_attention_on_image(image, attention_map):
    # attention_map: [14x14] for example
    attention_map = cv2.resize(attention_map, (image.width, image.height))
    attention_map = (attention_map - attention_map.min()) / (attention_map.max() - attention_map.min())
    
    # Convert image to array
    img_np = np.array(image)
    heatmap = cv2.applyColorMap((attention_map*255).astype(np.uint8), cv2.COLORMAP_JET)
    blended = cv2.addWeighted(img_np, 0.6, heatmap, 0.4, 0)
    
    plt.figure(figsize=(8,8))
    plt.imshow(blended)
    plt.axis('off')
    plt.show()


: 